### Importing Necessary Libraries

In [1]:
import numpy as np
import tensorflow as tf
import nibabel as nib
import os
import pickle
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Activation, BatchNormalization, Multiply, Add
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


C:\Users\muqsi\anaconda3\envs\brain\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


### Load and Preprocess Data (Using PSO)

In [2]:
import os
import numpy as np
import nibabel as nib
import tensorflow as tf
from sklearn.model_selection import train_test_split

def load_nii_images(directory, img_size=(128, 128)):
    images = []
    masks = []
    
    for folder in os.listdir(directory):
        folder_path = os.path.join(directory, folder)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                if file.endswith('.nii') or file.endswith('.nii.gz'):
                    img_path = os.path.join(folder_path, file)
                    img_data = nib.load(img_path).get_fdata()
                    
                    # Normalize image data
                    img_data = np.interp(img_data, (img_data.min(), img_data.max()), (0, 1))
                    
                    # Handle cases where mask is missing
                    if img_data.shape[-1] > 1:
                        img_slices = img_data[:, :, :-1]  # Image
                        mask_slice = img_data[:, :, -1]   # Last slice as mask
                    else:
                        img_slices = img_data
                        mask_slice = np.zeros_like(img_slices)  # Create blank mask if missing

                    # Resize images and masks
                    img_resized = np.array([
                        tf.image.resize(img_slices[:, :, i][..., np.newaxis], img_size).numpy()
                        for i in range(img_slices.shape[-1])
                    ])
                    
                    mask_resized = tf.image.resize(mask_slice[..., np.newaxis], img_size).numpy()
                    
                    # Append to dataset
                    images.append(img_resized)
                    masks.append(mask_resized)

    # Convert lists to numpy arrays
    images = np.array(images).reshape(-1, img_size[0], img_size[1], 1)
    masks = np.array(masks).reshape(-1, img_size[0], img_size[1], 1)

    return images, masks

# Load data
data_dir = "../data/brats-men-train"
X, y = load_nii_images(data_dir)

# Ensure equal shape before splitting
min_len = min(len(X), len(y))
X, y = X[:min_len], y[:min_len]

# Normalize masks for binary classification
y = (y > 0.5).astype("float32")

# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print dataset info
print("Dataset loaded successfully!")
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


Dataset loaded successfully!
X_train shape: (179, 128, 128, 1), y_train shape: (179, 128, 128, 1)
X_test shape: (45, 128, 128, 1), y_test shape: (45, 128, 128, 1)


In [3]:
# Save preprocessing pipeline
with open("../models/preprocessing.pkl", "wb") as f:
    pickle.dump({"X_train": X_train, "X_test": X_test, "y_train": y_train, "y_test": y_test}, f)


### Define Attention Block

In [4]:
def attention_block(x, g):
    theta_x = Conv2D(64, (1, 1), padding="same")(x)
    phi_g = Conv2D(64, (1, 1), padding="same")(g)
    phi_g = UpSampling2D(size=(2, 2), interpolation='bilinear')(phi_g)
    
    f = Activation("relu")(Add()([theta_x, phi_g]))
    psi_f = Conv2D(1, (1, 1), padding="same", activation="sigmoid")(f)
    
    return Multiply()([x, psi_f])


### Define Attention U-Net Model

In [5]:
def build_attention_unet(input_shape=(128, 128, 1)):
    inputs = Input(input_shape)
    
    # Encoder
    conv1 = Conv2D(64, (3, 3), activation="relu", padding="same")(inputs)
    pool1 = MaxPooling2D((2, 2))(conv1)
    
    conv2 = Conv2D(128, (3, 3), activation="relu", padding="same")(pool1)
    pool2 = MaxPooling2D((2, 2))(conv2)
    
    # Bottleneck
    bottleneck = Conv2D(256, (3, 3), activation="relu", padding="same")(pool2)
    
    # Decoder
    attn1 = attention_block(conv2, bottleneck)
    up1 = UpSampling2D((2, 2))(bottleneck)
    merge1 = concatenate([attn1, up1], axis=-1)
    conv3 = Conv2D(128, (3, 3), activation="relu", padding="same")(merge1)
    
    attn2 = attention_block(conv1, conv3)
    up2 = UpSampling2D((2, 2))(conv3)
    merge2 = concatenate([attn2, up2], axis=-1)
    conv4 = Conv2D(64, (3, 3), activation="relu", padding="same")(merge2)
    
    outputs = Conv2D(1, (1, 1), activation="sigmoid")(conv4)
    
    return Model(inputs, outputs)

model = build_attention_unet()
model.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])
model.summary()




Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 1)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 128, 128, 64)         640       ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 64, 64, 64)           0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv2d_1 (Conv2D)           (None, 64, 64, 128)          73856     ['max_pooling2d[0][0]'] 

### Train the Model with Early Stopping

In [ ]:
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20,
    batch_size=8,
    callbacks=[early_stopping]
)


Epoch 1/20


23/23 [==============================] - 87s 4s/step - loss: 0.6693 - accuracy: 0.9950 - val_loss: 0.5836 - val_accuracy: 1.0000
Epoch 2/20
 4/23 [====>.........................] - ETA: 1:09 - loss: 0.5518 - accuracy: 1.0000

### Saving the model

In [ ]:
# Save trained model
model.save("../models/attention_unet_model.h5")

### Testing

In [ ]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

# Load the segmentation file
file_path = r"C:\Users\RAHUL\Desktop\brain\data\brats-men-train\BraTS-MEN-00012-000\BraTS-MEN-00012-000-t1c.nii"  # Change this to the correct path
seg_img = nib.load(file_path).get_fdata()

# Select the middle slice for visualization
mid_slice = seg_img.shape[2] // 2
seg_slice = seg_img[:, :, mid_slice]

# Calculate tumor size (number of nonzero pixels)
tumor_pixels = np.sum(seg_slice > 0)  # Count tumor pixels
total_pixels = seg_slice.shape[0] * seg_slice.shape[1]  # Total pixels in the slice
tumor_percentage = (tumor_pixels / total_pixels) * 100  # Calculate percentage

# Display the segmentation mask with tumor percentage
plt.figure(figsize=(6, 6))
plt.title(f"Tumor Segmentation (Middle Slice)\nTumor Size: {tumor_pixels} pixels ({tumor_percentage:.2f}%)")
plt.imshow(seg_slice, cmap="jet")
plt.colorbar(label="Segmentation Labels")
plt.show()

# Print tumor size and percentage
print(f"Tumor Size: {tumor_pixels} pixels")
print(f"Tumor Percentage: {tumor_percentage:.2f}%")
